In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
import pandas as pd
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
def data_frame_adder(temp_frame,file_name):
    file_name=file_name.split('.csv')[0]
    temp_frame['file_name']=file_name
    temp_frame['subject']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[0]))
    temp_frame['session']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[1]))
    #temp_frame['type']=temp_frame['file_name'].apply(lambda x: x.split('__')[1])
    temp_frame = temp_frame.drop(columns=['file_name'])
    temp_frame = temp_frame[temp_frame.columns[::-1]]
    temp_frame = temp_frame.sort_values(by=['session','subject'], axis=0)
    col_list1 = list(temp_frame.columns)[0:2]
    col_list2 = list(temp_frame.columns)[2:]
    col_list2.sort()
    col_list=col_list1+col_list2
    
    #temp_frame = swap_columns(temp_frame, 'subject', 'session')
    # Swapping the columns
    temp_frame = temp_frame.reindex(columns=col_list)

    return temp_frame

In [2]:

Training_data_files=glob.glob('TrainingData/*.csv')
file_name_list_x=[]
file_name_list_x_time=[]
file_name_list_y=[]
file_name_list_y_time=[]
for file_name in tqdm(Training_data_files):
    if 'x.csv' in file_name:
        file_name_list_x.append(file_name)
    elif 'x_time.csv' in file_name:
        file_name_list_x_time.append(file_name)
    elif 'y.csv' in file_name:
        file_name_list_y.append(file_name)
    else:
        file_name_list_y_time.append(file_name)
file_name_list_x.sort()
file_name_list_x_time.sort()
file_name_list_y.sort()
file_name_list_y_time.sort()
Subject_list= [f.split('_')[1] for f in file_name_list_x]
Session_list= [f.split('_')[2] for f in file_name_list_x]

x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]
for sub,sess in tqdm(zip(Subject_list,Session_list)):
    x_sensor_dat=pd.concat([pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)
    features_to_normalize = [0,1,2,3,4,5]
    features_to_normalize = [0,1,2,3,4,5]
    scaler = StandardScaler()
    scaler.fit(x_sensor_dat[features_to_normalize])
    scaled = scaler.fit_transform(x_sensor_dat[features_to_normalize])
    scaled_df = pd.DataFrame(scaled, columns=x_sensor_dat[features_to_normalize].columns)
    x_sensor_dat[features_to_normalize]=scaled_df
    break

100%|█████████████████████████████████| 116/116 [00:00<00:00, 806864.45it/s]
0it [00:00, ?it/s]


In [137]:
x_sensor_dat = x_sensor_dat.applymap(lambda x: np.array([x]))

In [120]:
x_sensor_dat[0][0]

array([0.49225123])

In [78]:
new_chunk={'0':np.vstack((chunk[0]))[:,0],'1':np.vstack((chunk[1]))[:,0]}

In [125]:
x_sensor_dat[0]

0        [0.49225123220935507]
1        [0.45489722483915207]
2         [0.5086999002612168]
3         [0.5545241549751831]
4         [0.5033684694494693]
                 ...          
37885    [0.11504486918898627]
37886    [0.11544359469698251]
37887    [0.11185551634211324]
37888    [0.11098857789319193]
37889    [0.09791621697203544]
Name: 0, Length: 37890, dtype: object

In [154]:
x_sensor_dat.rolling(5).sum()

,0,1,2,3,4,5,time_stamp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.513741,0.355701,-2.941889,0.020852,-0.147346,0.043315,0.250
...,...,...,...,...,...,...,...
37885,0.520378,1.233310,-4.677776,-0.005161,-0.109236,0.079978,4735.375
37886,0.515866,1.231428,-4.692793,-0.003164,-0.106741,0.089824,4735.500
37887,0.515463,1.234133,-4.696992,-0.003088,-0.100985,0.104259,4735.625
37888,0.543550,1.228935,-4.710093,-0.003252,-0.092350,0.103975,4735.750


In [155]:
x_sensor_dat

,0,1,2,3,4,5,time_stamp
0,[[0.49225123220935507]],[[0.0574821663159993]],[[-0.5759895047349772]],[[0.0037880950409109005]],[[-0.04299058306919061]],[[0.00033990132530855053]],[[0.0]]
1,[[0.45489722483915207]],[[0.07778523441849579]],[[-0.5876824799481262]],[[0.001198472299551625]],[[-0.023188231591690773]],[[0.019269946529635873]],[[0.025]]
2,[[0.5086999002612168]],[[0.08656965800233246]],[[-0.5908608161680209]],[[0.0018721138559335082]],[[-0.00783203624064062]],[[0.025632387223924923]],[[0.05]]
3,[[0.5545241549751831]],[[0.08697396258912203]],[[-0.5889788440318348]],[[0.00794725289826185]],[[-0.0267234104263596]],[[0.003901962817739687]],[[0.075]]
4,[[0.5033684694494693]],[[0.046890343885634525]],[[-0.5983770876008141]],[[0.006045679136320407]],[[-0.04661176008987771]],[[-0.005829252660238366]],[[0.1]]
...,...,...,...,...,...,...,...
37885,[[0.11504486918898627]],[[0.24747590569891953]],[[-0.9340936692884059]],[[-0.0011393028237066374]],[[-0.011960546482248455]],[[0.0234732928178548]],[[947.125]]
37886,[[0.11544359469698251]],[[0.24446038180963306]],[[-0.9455903505129394]],[[0.0006331769809193811]],[[-0.017087109191913134]],[[0.02494373358668387]],[[947.15]]
37887,[[0.11185551634211324]],[[0.24346200259819348]],[[-0.9470934991893893]],[[-0.0009119555833883671]],[[-0.024020365887235478]],[[0.023949788414026854]],[[947.175]]
37888,[[0.11098857789319193]],[[0.24617501635502942]],[[-0.9471945328600001]],[[-0.0018576515205771392]],[[-0.023307326385002956]],[[0.013183799236134004]],[[947.2]]


In [ ]:
from more_itertools import sliced
CHUNK_SIZE = 5

index_slices = sliced(range(len(x_sensor_dat)), CHUNK_SIZE)
new_df=pd.DataFrame([])
for index_slice in index_slices:
    new_chunk=chunk[[0,1,2,3,4,5]][:1].reset_index(drop=True)
    chunk = x_sensor_dat.iloc[index_slice] # your dataframe chunk ready for use
    #chunk['time_stamp']=np.min(chunk['time_stamp'])
    for c in new_chunk.columns:
        chunk[c]=[np.vstack(chunk[c])[:,0]]*5
    chunk=chunk[0:1]
    new_df=pd.concat([new_df,chunk],axis=0)

In [170]:
new_df

,0,1,2,3,4,5,time_stamp
0,"[0.49225123220935507, 0.45489722483915207, 0.5...","[0.0574821663159993, 0.07778523441849579, 0.08...","[-0.5759895047349772, -0.5876824799481262, -0....","[0.0037880950409109005, 0.001198472299551625, ...","[-0.04299058306919061, -0.023188231591690773, ...","[0.00033990132530855053, 0.019269946529635873,...",[[0.0]]
5,"[0.4608525384902692, 0.5068086990022411, 0.492...","[0.06811913951240123, 0.08515794405089602, 0.0...","[-0.597594868729395, -0.5967253455014901, -0.6...","[0.0032723705678682764, 0.003870603228642579, ...","[-0.0203759064614629, -0.004201614006659609, -...","[0.0002196189704329571, 0.00832648580893115, 0...",[[0.125]]
10,"[0.503695150627654, 0.4415258573884231, 0.4165...","[0.06578867588235503, 0.05175226046484526, 0.0...","[-0.6030872984467123, -0.5889068531411032, -0....","[-0.001260552906553373, -0.003766024043053841,...","[-0.019746956485525772, -0.045101272687770574,...","[0.03972920796549599, 0.031103462770188904, 0....",[[0.25]]
15,"[0.4858648558249611, 0.5220234386996702, 0.545...","[0.06337338354035979, 0.057639236253608075, 0....","[-0.6013908480680589, -0.593958008621101, -0.6...","[-0.00010280893357242504, -0.00256305251927176...","[-0.011735594853288038, 0.002113631872944475, ...","[0.09861390524380124, 0.08099066128421872, 0.0...",[[0.375]]
20,"[0.5176103850858951, 0.5056475670061147, 0.530...","[0.06434617731771121, 0.046660622260856384, 0....","[-0.5931595961996887, -0.5903274851193262, -0....","[-0.0007252905375889715, -0.006914659766179086...","[-0.012118537412577393, -0.007256457470568876,...","[0.026809490070861947, 0.002903014611150704, -...",[[0.5]]
...,...,...,...,...,...,...,...
37865,"[0.11293994143175975, 0.09814663850287293, 0.1...","[0.24090455375343192, 0.2444389009906393, 0.24...","[-0.939374174721027, -0.9415745437331156, -0.9...","[4.2959313620474364e-05, 0.0001181090773989036...","[-0.023426716979456974, -0.029199540084929452,...","[0.03278747507435171, 0.0334321469613518, 0.03...",[[946.625]]
37870,"[0.11159366002018889, 0.12239369164115403, 0.1...","[0.24272850788084038, 0.24735180950256827, 0.2...","[-0.9348686714690669, -0.9404210701264138, -0....","[-0.001216747974161044, 6.983511761458905e-05,...","[-0.020563278725323448, -0.018285811340681545,...","[0.029640958461131798, 0.02841220936041057, 0....",[[946.75]]
37875,"[0.10897690166933309, 0.10407337507118383, 0.1...","[0.24587469535062922, 0.24880374868192898, 0.2...","[-0.943511045888736, -0.9450749731827155, -0.9...","[-0.00045645681275381926, -2.8895949281532762e...","[-0.023588989602761685, -0.023051947782605896,...","[0.02189725561305316, 0.017075666638607403, 0....",[[946.875]]
37880,"[0.1002792409087118, 0.11995576566638555, 0.11...","[0.245541126963965, 0.24634303193289, 0.240757...","[-0.9482730408912434, -0.930573332333325, -0.9...","[-0.001599529845561789, -0.0013636953478428052...","[-0.01654573828207352, -0.019582328302935246, ...","[0.018515648733647277, 0.015097548860245722, 0...",[[947.0]]


In [58]:
chunk['time_stamp']=[np.min(chunk['time_stamp'])]*5

/var/folders/g4/wndq11z17l7bl56tk3j1s6980000gn/T/ipykernel_24816/3686919733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['time_stamp']=[np.min(chunk['time_stamp'])]*5


In [55]:
new_chunk[c]

0    0.100279
Name: 0, dtype: float64

In [35]:
new_chunk[2]

0   -0.934094
Name: 2, dtype: float64

In [28]:

    print(c)

0
1
2
3
4
5


array([[0.11504487],
       [0.11544359],
       [0.11185552],
       [0.11098858],
       [0.09791622]])

In [12]:
np.vstack(chunk)

array([['0'],
       ['1'],
       ['2'],
       ['3'],
       ['4'],
       ['5'],
       ['time_stamp']], dtype='<U21')

In [ ]:

Training_data_files=glob.glob('TrainingData/*.csv')
file_name_list_x=[]
file_name_list_x_time=[]
file_name_list_y=[]
file_name_list_y_time=[]
for file_name in tqdm(Training_data_files):
    if 'x.csv' in file_name:
        file_name_list_x.append(file_name)
    elif 'x_time.csv' in file_name:
        file_name_list_x_time.append(file_name)
    elif 'y.csv' in file_name:
        file_name_list_y.append(file_name)
    else:
        file_name_list_y_time.append(file_name)
file_name_list_x.sort()
file_name_list_x_time.sort()
file_name_list_y.sort()
file_name_list_y_time.sort()
Subject_list= [f.split('_')[1] for f in file_name_list_x]
Session_list= [f.split('_')[2] for f in file_name_list_x]

x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]
for sub,sess in tqdm(zip(Subject_list,Session_list)):
    x_sensor_dat=pd.concat([pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)
    features_to_normalize = [0,1,2,3,4,5]
    features_to_normalize = [0,1,2,3,4,5]
    scaler = StandardScaler()
    scaler.fit(x_sensor_dat[features_to_normalize])
    scaled = scaler.fit_transform(x_sensor_dat[features_to_normalize])
    scaled_df = pd.DataFrame(scaled, columns=x_sensor_dat[features_to_normalize].columns)
    x_sensor_dat[features_to_normalize]=scaled_df
    x_sensor_dat['measurements']=x_sensor_dat.apply(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5]],axis=1)
    x_sensor_dat=x_sensor_dat[['time_stamp','measurements']]
    x_sensor_dat['Subject']=int(sub)
    x_sensor_dat['Session']=int(sess)
    
    y_sensor_dat=pd.concat([pd.read_csv(f"TrainingData/subject_{sub}_{sess}__y.csv",header=None),pd.read_csv(f"TrainingData/subject_{sub}_{sess}__y_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1).rename(columns={0:'labels'})
    y_sensor_dat=y_sensor_dat[['time_stamp','labels']]
    trp=pd.concat(
        [pd.DataFrame({'labels':0},index=[x_sensor_dat.index[-1]]),pd.DataFrame({'time_stamp':np.max(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[-1]])
                         ]
         ,axis=1)
    y_sensor_dat=pd.concat([y_sensor_dat,trp],axis=0)
    trp=pd.concat([pd.DataFrame({'labels':0},index=[x_sensor_dat.index[0]]),pd.DataFrame({'time_stamp':np.min(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[0]])],axis=1)
    y_sensor_dat=pd.concat([trp,y_sensor_dat],axis=0)
    # y_sensor_dat=y_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    #y_sensor_dat['Subject']=int(sub)
    #y_sensor_dat['Session']=int(sess)
    x_sensor_dat["time_stamp"]=pd.to_datetime(x_sensor_dat["time_stamp"],unit='s').round('ms')
    x_sensor_dat=x_sensor_dat.set_index('time_stamp')
    x_sensor_dat=x_sensor_dat.asfreq('0.025S')
    
    y_sensor_dat["time_stamp"]=pd.to_datetime(y_sensor_dat["time_stamp"],unit='s').round('ms')
    y_sensor_dat=y_sensor_dat.set_index('time_stamp')
    y_sensor_dat=y_sensor_dat.asfreq(freq='0.025S', method='bfill')
    x_y_dat=pd.merge(x_sensor_dat,y_sensor_dat, how='inner', left_index=True, right_index=True)
    x_y_dat.to_parquet(f"pre_processed_training/subject_{sub}_session_{sess}.gzip",compression='gzip')


In [ ]:
files=glob.glob('pre_processed_training/*.gzip')
total_training_data=pd.DataFrame([])
for f in tqdm(files):
    total_training_data=pd.concat([total_training_data,pd.read_parquet(f, engine='auto')],axis=0)
total_training_data=total_training_data.sort_values(['Subject','Session','time_stamp'])
total_training_data=total_training_data.reset_index()
total_training_data['epoch'] = total_training_data['time_stamp'].sub(pd.Timestamp('1970-01-01 00:00:00.000'))
total_training_data['epoch']=total_training_data['epoch'].dt.total_seconds()
total_training_data=total_training_data[['Subject','Session','epoch','measurements','labels']]
total_training_data_grouped=total_training_data.groupby(['Subject','Session','epoch']).agg({'measurements':[np.mean],'labels':[np.mean]})
total_training_data_grouped.columns = total_training_data_grouped.columns.droplevel(1)
total_training_data_grouped=total_training_data_grouped.reset_index(level=['epoch'])
total_training_data_grouped.to_parquet(f"combined_sampled/training.gzip",compression='gzip')